For more information on using spot instances to train deep learning models see [Train Deep Learning Models on GPUs using Amazon EC2 Spot Instances](https://aws.amazon.com/blogs/machine-learning/train-deep-learning-models-on-gpus-using-amazon-ec2-spot-instances/)

In [ ]:
import os
import sys

import boto3

# Add the mlops directory to the PYTHONPATH
sys.path.insert(0, os.path.abspath("../.."))

from mlops.aws.ec2 import available_zones
from mlops.key_pair import read_pem_public_key, write_pem_private_key
from mlops.terraform_cli import variables_dict_to_options, variables_dict_to_tfvars

%load_ext autoreload
%autoreload 2

In [ ]:
DEFAULT_INSTANCE_TYPE = "p2.xlarge"
session = boto3.session.Session()
DEFAULT_REGION = session.region_name or "us-east-1"
DEFAULT_TAG_NAME = "mpsi"  # Acronym for magic packet spot instance
DEFAULT_KEY_NAME = DEFAULT_TAG_NAME
DEFAULT_KEY_PATH = f"~/.ssh/{DEFAULT_KEY_NAME}.pem"

# (Optional) Create an RSA key pair

The key pair is used to SSH into the EC2 instance. For more on Amazon EC2 key pairs see [Amazon EC2 key pairs and Linux instances](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html).

In [ ]:
key_path = (
    input(f"Enter the RSA private key path (or {DEFAULT_KEY_PATH}): ")
    or DEFAULT_KEY_PATH
)
write_pem_private_key(
    key_path,
    input('Enter the passhprase (or ""): '),
)

Make sure to set the proper permissions on the key file ([Connect to your Linux instance using SSH](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AccessingInstancesLinux.html))

In [ ]:
!chmod 400 $key_path

# Specify terraform input variables

In [ ]:
# These variables are specified in the *.tf files
variables = {
    "availability_zone": "",
    "instance_type": DEFAULT_INSTANCE_TYPE,
    "public_key": "",
    "key_name": DEFAULT_KEY_NAME,
    "region": DEFAULT_REGION,
    "tag_name": DEFAULT_TAG_NAME,
}

In [ ]:
# Provide instance type
variables["instance_type"] = (
    input(f"Enter the instance type (or {DEFAULT_INSTANCE_TYPE}): ")
    or DEFAULT_INSTANCE_TYPE
)

In [ ]:
# Retrieve public key from private key file
variables["public_key"] = read_pem_public_key(
    input(f"Enter the RSA private key path (or {DEFAULT_KEY_PATH}): ")
    or DEFAULT_KEY_PATH
)

In [ ]:
# Provide key name
variables["key_name"] = (
    input(f"Enter the key name (or {DEFAULT_KEY_NAME}): ") or DEFAULT_KEY_NAME
)

In [ ]:
# Provide AWS region
variables["region"] = (
    input(f"Enter the AWS region (or {DEFAULT_REGION}): ") or DEFAULT_REGION
)

In [ ]:
# Provide the availability zone
ec2 = session.client("ec2", region_name=variables["region"])
zones = available_zones(ec2)
print(f"Available zones: {zones}")
default_availability_zone = zones[0]
variables["availability_zone"] = (
    input(f"Enter the availability zone (or {default_availability_zone}): ")
    or default_availability_zone
)

In [ ]:
# Provide tag name
variables["tag_name"] = (
    input(f"Enter the tag name (or {DEFAULT_TAG_NAME}): ") or DEFAULT_TAG_NAME
)

# Terraform CLI

In [ ]:
# Creating options string for use in CLI commands
options = variables_dict_to_options(variables)
options

The `plan` subcommand is useful to sanity check what infrastructure terraform will modify in `apply`

In [ ]:
!terraform plan $options

`apply` will initiate the spot instance request. Afterwards, if the request is accepted by AWS, an EC2 instance will be accessible over SSH. For more on spot instances see [Spot Instances](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-spot-instances.html).

Once the EC2 instance is running it may connected to over SSH ([Connect to your Linux instance using SSH](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AccessingInstancesLinux.html)):

```sh
ssh -i /path/my-key-pair.pem my-instance-user-name@my-instance-public-dns-name
```

In [ ]:
!terraform apply -auto-approve $options

Once finished with the EC2 instance it is important to run the `destroy` subcommand to cancel the spot instance request. Otherwise, the request may stay open and AWS will try and fulfill it.

In [ ]:
!terraform destroy -auto-approve $options

## Save the variables to a variables file

Optionally, we may save the variable values to a `.tfvars` file for subsequent access. *Keep in mind* some of the AWS values, such as spot instance, are ephemeral. If the file is named `terraform.tfvars` or ends in `.auto.tfvars` then it will automatically be loaded by the CLI.

In [ ]:
variables_dict_to_tfvars(variables)

In [ ]:
!cat terraform.tfvars

In [ ]:
!terraform plan

In [ ]:
!terraform apply -auto-approve

In [ ]:
!terraform destroy -auto-approve